In [3]:
import os
import time

import pandas as pd
import polars as pl
import yfinance as yf
from dotenv import load_dotenv
from investment_info_fetcher import investment_info_fetcher

# PostgreSQLへの接続情報
load_dotenv()
DB_HOST: str | None = os.getenv("DB_HOST")
DB_PORT: str | None = os.getenv("DB_PORT")
DB_USER: str | None = os.getenv("DB_USER")
DB_PASSWORD: str | None = os.getenv("DB_PASSWORD")
DB_NAME: str | None = os.getenv("DB_NAME")

In [ ]:
# データを取得
def historical_data_fetcher(df_investment_info: pl.DataFrame) -> tuple[pl.DataFrame, pl.DataFrame]:
    try:
        # ティッカーシンボルのリスト
        ticker_symbol_list: list[str] = df_investment_info.select(pl.col("ticker_symbol")).to_numpy().flatten().tolist()

        # yFinance を使って株価データを取得
        # yf_tickers = yf.Tickers(tickers=ticker_symbol_list)
        # historical_data = yf_tickers.history(period="max")  # TODO 期間を変更する

        # 分割してリクエスト
        # TODO listはだめ
        chunk_size = 200
        historical_dataframe_list: list[pd.DataFrame] = []
        for i in range(0, len(ticker_symbol_list), chunk_size):
            chunk = ticker_symbol_list[i : i + chunk_size]
            yf_tickers = yf.Tickers(tickers=chunk)
            try:
                historical_data_chunk: pd.DataFrame = yf_tickers.history(period="max")[["Close", "Dividends"]]
                historical_dataframe_list.append(historical_data_chunk)
                time.sleep(1)
            except Exception as e:
                print(f"データの取得に失敗しました: {e}")
                time.sleep(5)
        historical_data: pd.DataFrame = pd.concat(historical_dataframe_list, axis=1)
        print(historical_data.info())

        # 日付をpolarsのDataFrameに変換
        # 2つの戻り値があるのは，仕方ない．
        df_date: pl.DataFrame = (
            pl.DataFrame(historical_data.index.to_numpy())
            .rename({"column_0": "date"})
            .with_columns(pl.col("date").cast(pl.Date))
        )

        print("データの取得に成功しました")
        return historical_data, df_date, historical_dataframe_list

    except Exception as e:
        print(f"データの取得に失敗しました: {e}")
        exit()


In [4]:
df_investment_info: pl.DataFrame = investment_info_fetcher(DB_HOST, DB_PORT, DB_USER, DB_PASSWORD, DB_NAME)

In [16]:
historical_data, df_time, historical_dataframe_list = historical_data_fetcher(df_investment_info)

[*********************100%***********************]  200 of 200 completed
[*********************100%***********************]  100 of 100 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 13877 entries, 1996-03-28 to 2025-03-10
Columns: 600 entries, ('Close', 'AADR') to ('Dividends', 'YXI')
dtypes: float64(600)
memory usage: 63.6 MB
None
データの取得に成功しました


In [23]:
historical_data.describe()

Price         Close                                                    \
Ticker         AADR         ACWX       AGEM          AMJ         ANGL   
count   3682.000000  4263.000000  14.000000  3835.000000  3247.000000   
mean      42.609727    36.621766  29.498928    17.566692    20.606838   
std       12.428945     8.904226   0.478172     4.506697     4.760751   
min       21.780319    14.589430  28.643999     5.115770    12.150651   
25%       32.931009    30.427525  29.276249    15.166248    15.852170   
50%       42.461906    35.139824  29.516500    17.350647    20.496456   
75%       52.263247    42.444422  29.893250    20.532530    24.684964   
max       78.440002    56.994923  30.209999    28.950001    29.030001   

Price                                                                   ...  \
Ticker        AXJS         BDCL         BKLN          BNO          BOS  ...   
count   973.000000  2251.000000  3526.000000  3716.000000  4069.000000  ...   
mean     56.844573    19.570818    15.823589    25.987604    21.325482  ...   
std       2.899447     5.375108     2.033610    10.301602     4.399191  ...   
min      51.450001     0.893380    11.758349     6.010000    13.820100  ...   
25%      54.869999    15.559950    14.357674    17.340000    19.129999  ...   
50%      56.990002    18.780001    15.639171    25.407500    19.129999  ...   
75%      58.410000    23.804950    17.198723    34.070000    22.360001  ...   
max      64.250000    30.590000    20.967787    46.450001    47.639999  ...   

Price     Dividends                                                         \
Ticker          VWO         VWOB    WEET    WMCR  WMH    WREI          XLV   
count   5032.000000  2960.000000  1756.0  3128.0  2.0  2257.0  6593.000000   
mean       0.004204     0.013727     0.0     0.0  0.0     0.0     0.003714   
std        0.048694     0.062166     0.0     0.0  0.0     0.0     0.036302   
min        0.000000     0.000000     0.0     0.0  0.0     0.0     0.000000   
25%        0.000000     0.000000     0.0     0.0  0.0     0.0     0.000000   
50%        0.000000     0.000000     0.0     0.0  0.0     0.0     0.000000   
75%        0.000000     0.000000     0.0     0.0  0.0     0.0     0.000000   
max        1.178000     0.382000     0.0     0.0  0.0     0.0     0.666000   

Price                                          
Ticker         XMPT          XTN          YXI  
count   3435.000000  3550.000000  3768.000000  
mean       0.004923     0.001923     0.000369  
std        0.022519     0.019895     0.006850  
min        0.000000     0.000000     0.000000  
25%        0.000000     0.000000     0.000000  
50%        0.000000     0.000000     0.000000  
75%        0.000000     0.000000     0.000000  
max        0.180000     0.683000     0.220000  

[8 rows x 600 columns]

In [34]:
df_value = (
    pl.DataFrame(historical_data['Close'].reset_index())
    .with_columns(pl.col("Date").cast(pl.Date))
    .rename({"Date": "date"})
)
df_value.head()

date,AADR,ACWX,AGEM,AMJ,ANGL,AXJS,BDCL,BKLN,BNO,BOS,BRAQ,BRAZ,BRIS,BSCF,BSCK,BSV,BXUC,CHII,CHIX,CHNA,CNDA,COW,CRO,CUT,CVRT,CZM,DAG,DBB,DBE,DBT,DEW,DFJ,DGLD,DGZ,DLN,DOY,…,PSTL,QQQ,REZ,RMS,ROOF,RTG,RWX,RZV,SAA,SCHD,SCHG,SCHV,SDP,SDYL,SMIN,SOCL,TDTT,TTFS,TWM,TYO,UCD,UGAZ,UGE,VEA,VTHR,VTIP,VTWO,VWO,VWOB,WEET,WMCR,WMH,WREI,XLV,XMPT,XTN,YXI
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1996-03-28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-03-29,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-04-01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-04-02,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-04-03,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [33]:
df_dividend_pivot = (
    pl.DataFrame(historical_data['Dividends'].reset_index())
    .with_columns(pl.col("Date").cast(pl.Date))
    .rename({"Date": "date"})
)
df_dividend_pivot.head()

date,AADR,ACWX,AGEM,AMJ,ANGL,AXJS,BDCL,BKLN,BNO,BOS,BRAQ,BRAZ,BRIS,BSCF,BSCK,BSV,BXUC,CHII,CHIX,CHNA,CNDA,COW,CRO,CUT,CVRT,CZM,DAG,DBB,DBE,DBT,DEW,DFJ,DGLD,DGZ,DLN,DOY,…,PSTL,QQQ,REZ,RMS,ROOF,RTG,RWX,RZV,SAA,SCHD,SCHG,SCHV,SDP,SDYL,SMIN,SOCL,TDTT,TTFS,TWM,TYO,UCD,UGAZ,UGE,VEA,VTHR,VTIP,VTWO,VWO,VWOB,WEET,WMCR,WMH,WREI,XLV,XMPT,XTN,YXI
date,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1996-03-28,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-03-29,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-04-01,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-04-02,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
1996-04-03,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [25]:
historical_dataframe_list: list[pd.DataFrame] = []

historical_data_chunk = pd.DataFrame({
    "date": ["2021-01-01", "2021-01-02", "2021-01-03"],
    "Close, A": [100, 101, 102],
    "Dividends, A": [1, 2, 3],
    "Close, C": [239, 240, 241],
    "Dividends, C": [7, 8, 9]
}).set_index("date")

historical_dataframe_list.append(historical_data_chunk)

historical_data_chunk = pd.DataFrame({
    "date": ["2021-01-03", "2021-01-04", "2021-01-05"],
    "Close, B": [103, 104, 105],
    "Dividends, B": [4, 5, 6],
    "Close, D": [242, 243, 244],
    "Dividends, D": [10, 11, 12]
}).set_index("date")

historical_dataframe_list.append(historical_data_chunk)

historical_data = pd.concat(historical_dataframe_list, axis=1)
historical_data

,"Close, A","Dividends, A","Close, C","Dividends, C","Close, B","Dividends, B","Close, D","Dividends, D"
date,,,,,,,,
2021-01-01,100.0,1.0,239.0,7.0,NaN,NaN,NaN,NaN
2021-01-02,101.0,2.0,240.0,8.0,NaN,NaN,NaN,NaN
2021-01-03,102.0,3.0,241.0,9.0,103.0,4.0,242.0,10.0
2021-01-04,NaN,NaN,NaN,NaN,104.0,5.0,243.0,11.0
2021-01-05,NaN,NaN,NaN,NaN,105.0,6.0,244.0,12.0
